In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
de2engl = {"Montag": "Monday", "Dienstag": "Tuesday", "Mittwoch": "Wednesday",
           "Donnerstag": "Thursday", "Freitag": "Friday", "Samstag": "Saturday", "Sonntag": "Sunday"}
dictinary_list = []

for league in [1, 2]:
    if league == 1:
        R = range(63, 120)  # Year = 1900+k
    if league == 2:
        R = range(74, 120)  # Year = 1900+k

    for k in R:
        # First load a website to just see how many matchdays there are
        prefix = 19 if k < 100 else 20
        url = ('http://www.kicker.de/news/fussball/bundesliga/spieltag/'
               '{}-bundesliga/{}{:02d}-{:02d}/-1/0/spieltag.html'.format(league, prefix, k % 100, (k+1) % 100))
        page = requests.get(url)
        print(url)
        soup = BeautifulSoup(page.text, 'html.parser')  # Get website content
        matchdaytag = "kick__v100-gameList kick__module-margin"
        matchdays = soup.findAll("div", {"class": matchdaytag})

        for matchday in range(1, 1+len(matchdays)):  # The roughly 30 matchdays through the year
            url = "https://www.kicker.de/{}-bundesliga/spieltag/{}{:02d}-{:02d}/{}".format(
                league, prefix, k % 100, (k+1) % 100, matchday)
            page = requests.get(url)
            soup = BeautifulSoup(page.text, 'html.parser')  # Get website content

            matchdatestag = "kick__v100-gameList kick__module-margin"
            matchdates = soup.findAll("div", {"class": matchdatestag})
            for matchdate in matchdates:  # The weekday during which the matches take place
                weekdaytag = "kick__v100-gameList__header"
                st = matchdate.findAll("div", {"class": weekdaytag})[0].text.strip().split(",")[1]
                date = pd.to_datetime(st, format = "%d.%m.%Y",errors='coerce').date()
                dayofweek = matchdate.findAll("div", {"class": weekdaytag})[
                    0].text.strip().split(",")[0]
                matchtag = "kick__v100-gameList__gameRow"
                games = matchdate.findAll("div", {"class": matchtag})
                for game in games:  # All the games on a Friday, Saturday, ...
                    teamstag = "kick__v100-gameCell__team__name"
                    teams = game.findAll("div", {"class": teamstag})
                    resulttag = "kick__v100-scoreBoard kick__v100-scoreBoard--standard"
                    goalstag = "kick__v100-scoreBoard__scoreHolder__score"
                    goals = game.findAll("a", {"class": resulttag})[
                        0].findAll("div", {"class": goalstag})

                    aborted = False
                    annull = False
                    try:
                        home = teams[0].text.strip()
                        guest = teams[1].text.strip()
                        goalshome = int(goals[0].text)
                        goalsguest = int(goals[1].text)
                        goalshome_halftime = int(goals[2].text)
                        goalsguest_halftime = int(goals[3].text)
                    except:
                        # Two games got aborted and repeated. We do not have a half time score for these.
                        # We do store them in the databse, but set the "aborted" flag to be true.
                        # The games are found here:
                        # https://www.kicker.de/bundesliga/spieltag/1963-64/14/0
                        # https://www.kicker.de/bundesliga/spieltag/2007-08/28/0
                        # For four more games, the half time score was not available.

                        try:
                            abortedtag = "kick__v100-scoreBoard__scoreHolder__text"
                            status = game.findAll("div", {"class": abortedtag})
                            aborted = status[0].text == "abgr."
                        except:
                            pass

                        try:
                            annultag = "kick__v100-scoreBoard__dateHolder"
                            status = game.findAll("div", {"class": abortedtag})
                            annull = status[0].text == "annull."
                        except:
                            pass

                        goalshome_halftime = pd.NA
                        goalsguest_halftime = pd.NA

                    # In 1995/96 the 3 points rule for winner was introduced.
                    pointswinner = 3 if k >= 95 else 2
                    if goalshome == goalsguest:
                        pointshome, pointsguest = 1, 1
                    elif goalshome > goalsguest:
                        pointsguest, pointshome = pointswinner, 0
                    else:
                        pointsguest, pointshome = 0, pointswinner

                    entry = {'SeasonFrom': 1900+k,
                             'SeasonTo': 1901+k,
                             'Matchday': matchday,
                             'Day': de2engl[dayofweek],
                             'Date': date,
                             'Time': pd.NA,
                             'Home': home,
                             'Guest': guest,
                             'Score90': "{}:{}".format(goalshome, goalsguest),
                             'Score45': "{}:{}".format(goalshome_halftime, goalsguest_halftime),
                             'Score90Home': goalshome,
                             'Score90Guest': goalsguest,
                             'Score45Home': goalshome_halftime,
                             'Score45Guest': goalsguest_halftime,
                             'PointsHome': pointshome,
                             'PointsGuest': pointsguest,
                             'Aborted': aborted,
                             'Annulled': annull,
                             'League': "{}. Bundesliga".format(league)}
                    dictinary_list.append(entry)
df = pd.DataFrame.from_dict(dictinary_list)
df.to_csv("AllBundesligaGamesEver.csv", sep=";")

http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1963-64/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1964-65/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1965-66/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1966-67/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1967-68/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1968-69/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1969-70/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1970-71/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1971-72/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spieltag/1-bundesliga/1972-73/-1/0/spieltag.html
http://www.kicker.de/news/fussball/bundesliga/spie